In [ ]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

def extract_labels_from_directories(filepaths):
    # Extract labels from directory names (assuming directory names are the class labels)
    return [os.path.basename(os.path.dirname(filepath)) for filepath in filepaths]

# Load image file paths
train_image_paths = [os.path.join(root, fname) 
                     for root, _, files in os.walk(os.path.join("dataset", "train")) 
                     for fname in files if fname.endswith('.jpg')]
test_image_paths = [os.path.join(root, fname) 
                    for root, _, files in os.walk(os.path.join("dataset", "test")) 
                    for fname in files if fname.endswith('.jpg')]
valid_image_paths = [os.path.join(root, fname) 
                     for root, _, files in os.walk(os.path.join("dataset", "valid")) 
                     for fname in files if fname.endswith('.jpg')]

print("Number of training images: ", len(train_image_paths))
# Extract labels from directory names
from sklearn.preprocessing import LabelEncoder

# Encode labels as integers
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(extract_labels_from_directories(train_image_paths))
test_labels = label_encoder.transform(extract_labels_from_directories(test_image_paths))
valid_labels = label_encoder.transform(extract_labels_from_directories(valid_image_paths))

print("Number of training labels: ", len(train_labels))
# Load images into numpy arrays
# train_images = np.array([plt.imread(path) for path in train_image_paths])
test_images = np.array([plt.imread(path) for path in test_image_paths])
# valid_images = np.array([plt.imread(path) for path in valid_image_paths])

# print("image shape: ", train_images.shape)

# Normalize the input image so that each pixel value is between 0 to 1.
# train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

# Apply data augmentation to the training images
# data_augmentation = tf.keras.Sequential([
#     tf.keras.layers.RandomFlip("horizontal_and_vertical"),
#     tf.keras.layers.RandomRotation(0.2),
#     tf.keras.layers.RandomZoom(0.2),
#     tf.keras.layers.RandomContrast(0.2)
# ])

# # Augment the training images
# train_images = np.array([data_augmentation(image[np.newaxis, ...])[0].numpy() for image in train_images])

# print("image shape after normalization: ", train_images.shape)
print("image shape after normalization: ", test_images.shape)
# print("image shape after normalization: ", valid_images.shape)
# print("labels shape: ", train_labels.shape)
# print("labels shape: ", test_labels.shape)
# print("labels shape: ", valid_labels.shape)

regularizer = tf.keras.regularizers.l2(0.01)


model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.25), 
    tf.keras.layers.Dense(53, kernel_regularizer=regularizer) 
])

train_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    # rotation_range=10,  # Rotate images up to 10 degrees
    # width_shift_range=0.1,  # Shift images horizontally by 10% of the width
    # height_shift_range=0.1,  # Shift images vertically by 10% of the height
    # shear_range=0.1,  # Shear transformations
    # zoom_range=0.1,  # Zoom in/out by 10%
    horizontal_flip=True,  # Flip images horizontally
    vertical_flip=True,  # Flip images vertically
    fill_mode='nearest'  # Fill in missing pixels after transformations
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': train_image_paths, 'class': train_labels}),
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=True
)

valid_datagen = ImageDataGenerator(rescale=1.0/255.0)
valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': valid_image_paths, 'class': valid_labels}),
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=False
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': test_image_paths, 'class': test_labels}),
    x_col='filename',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=False
)

model.compile(optimizer='adam',
                            loss=tf.keras.losses.SparseCategoricalCrossentropy(
                                    from_logits=True),
                            metrics=['accuracy'])


2025-04-17 13:48:10.526649: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-17 13:48:10.541692: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744894090.561589   80286 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744894090.566383   80286 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-17 13:48:10.585224: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Number of training images:  7624
Number of training labels:  7624
image shape after normalization:  (265, 224, 224, 3)


/home/said/anaconda3/envs/cpu_env/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
I0000 00:00:1744894093.203085   80286 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4269 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 7624 validated image filenames.
Found 265 validated image filenames.
Found 265 validated image filenames.


In [2]:
print("Training the model...")
model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=40,
    verbose=2,
)

score = model.evaluate(test_images, test_labels, verbose=1)
print("Test loss {:.4f}, accuracy {:.2f}%".format(score[0], score[1] * 100))



Training the model...
Epoch 1/40


/home/said/anaconda3/envs/cpu_env/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1744894094.875989   80542 service.cc:148] XLA service 0x7538bc003ee0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744894094.876028   80542 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2025-04-17 13:48:14.901738: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744894095.019257   80542 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-17 13:48:15.677718:

239/239 - 25s - 105ms/step - accuracy: 0.2385 - loss: 3.3534 - val_accuracy: 0.4868 - val_loss: 1.8638
Epoch 2/40
239/239 - 12s - 52ms/step - accuracy: 0.4811 - loss: 2.1255 - val_accuracy: 0.6679 - val_loss: 1.3414
Epoch 3/40
239/239 - 12s - 51ms/step - accuracy: 0.6142 - loss: 1.6132 - val_accuracy: 0.7736 - val_loss: 1.0396
Epoch 4/40
239/239 - 12s - 52ms/step - accuracy: 0.6889 - loss: 1.2904 - val_accuracy: 0.8000 - val_loss: 0.8736
Epoch 5/40
239/239 - 12s - 51ms/step - accuracy: 0.7516 - loss: 1.0562 - val_accuracy: 0.8264 - val_loss: 0.8373
Epoch 6/40
239/239 - 12s - 52ms/step - accuracy: 0.7882 - loss: 0.9182 - val_accuracy: 0.8415 - val_loss: 0.7295
Epoch 7/40
239/239 - 12s - 52ms/step - accuracy: 0.8219 - loss: 0.7685 - val_accuracy: 0.8302 - val_loss: 0.8224
Epoch 8/40
239/239 - 12s - 51ms/step - accuracy: 0.8480 - loss: 0.6597 - val_accuracy: 0.8302 - val_loss: 0.7430
Epoch 9/40
239/239 - 12s - 51ms/step - accuracy: 0.8712 - loss: 0.5865 - val_accuracy: 0.8604 - val_loss: 

In [ ]:
with tf.device('/GPU:0'):
    def representative_dataset():
        for i, data in enumerate(train_generator):
            if i >= 50:
                break
            # Use only the input data (X), not the labels (y)
            yield [data[0][:1]]  # Take one sample at a time

    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    # try to use unsigned int
    converter.inference_input_type = tf.uint8  # Input type for OpenMV
    converter.inference_output_type = tf.uint8  # Output type for OpenMV

    tflite_micro_model = converter.convert()

    tflite_micro_model_path = "playing_card_model_micro.tflite"
    with open(tflite_micro_model_path, "wb") as f:
        f.write(tflite_micro_model)

    model_size = len(tflite_micro_model) / 1024  # Size in KB
    print(f"Model post quantization size: {model_size:.2f} KB")

print(f"Micro TFLite model has been quantized and saved to {tflite_micro_model_path}")

INFO:tensorflow:Assets written to: /tmp/tmpkv4gl5a2/assets


INFO:tensorflow:Assets written to: /tmp/tmpkv4gl5a2/assets


Saved artifact at '/tmp/tmpkv4gl5a2'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 53), dtype=tf.float32, name=None)
Captures:
  128892149771856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128892149775376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128892150108176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128892150110112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128892150111168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128892150112048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128892150112576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128892150113456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128892150113984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128892150114864: TensorSpec(shape=(), dtype=tf.resource, name=None)


/home/said/anaconda3/envs/cpu_env/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1744894600.450965   80286 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1744894600.450981   80286 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-04-17 13:56:40.451300: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpkv4gl5a2
2025-04-17 13:56:40.451831: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-04-17 13:56:40.451846: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpkv4gl5a2
I0000 00:00:1744894600.455832   80286 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-04-17 13:56:40.456620: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-04-1

Model post quantization size: 21754.92 KB
Micro TFLite model has been quantized and saved to playing_card_model_micro.tflite


fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8


In [8]:
folder_names = ", ".join(sorted(next(os.walk("dataset/train"))[1]))
print(folder_names)

ace of clubs, ace of diamonds, ace of hearts, ace of spades, eight of clubs, eight of diamonds, eight of hearts, eight of spades, five of clubs, five of diamonds, five of hearts, five of spades, four of clubs, four of diamonds, four of hearts, four of spades, jack of clubs, jack of diamonds, jack of hearts, jack of spades, joker, king of clubs, king of diamonds, king of hearts, king of spades, nine of clubs, nine of diamonds, nine of hearts, nine of spades, queen of clubs, queen of diamonds, queen of hearts, queen of spades, seven of clubs, seven of diamonds, seven of hearts, seven of spades, six of clubs, six of diamonds, six of hearts, six of spades, ten of clubs, ten of diamonds, ten of hearts, ten of spades, three of clubs, three of diamonds, three of hearts, three of spades, two of clubs, two of diamonds, two of hearts, two of spades
